# Universidad Politécnica Salesiana
## Autor: Yandry Romero
## Investigar como pasar parametros y generar reportes utilizando Notebook

Papermill es una herramienta que nos permite parametrizar y ejecutar cuadernos. Transforma su cuaderno Jupyter en una herramienta de flujo de trabajo de datos, ejecutando cada celda secuencialmente, sin tener que abrir JupyterLab (o Notebook). Intenta llenar el vacío de la automatización y el registro al ofrecernos una forma de ejecutar cuadernos como archivos y también al generar un informe para cada ejecución.

Papermill te permite:

    - Parametrizar cuadernos

    - Ejecutar cuadernos

#   Configuración de nuestro entorno de desarrollo

Comenzaremos creando un entorno de desarrollo usando Conda (1, 2) e instalando JupyterLab y otras bibliotecas utilizadas en el análisis (3) .

# 1) Cree un entorno
 conda conda create -n papermill python = 3.7
# 2)Actívalo 
conda activa la papelera
# 3) Instale las bibliotecas usando pip (o conda)
 pip install papermill pyowm jupyterlab pandas seaborn boto3 pdfkit

Después de instalar Papermill podemos obtener más información a través del terminal.

# 4) Papermill Documentation
papermill -h

# Instalación del núcleo de Jupyter

Aunque los cuadernos de Jupyter son famosos por ejecutar Python, podemos usar prácticamente cualquier lenguaje de programación instalando diferentes núcleos . Con un kernel específico, podemos ejecutar nuestros cuadernos en entornos definidos con Papermill, evitando problemas con bibliotecas faltantes (5) .

# 5) Instale el kernel de Jupyter para el entorno de la fábrica de papel

 pip install ipykernel 
ipython kernel install --user --name = papermill-tutorial

# Crear un flujo de trabajo
Vamos a utilizar un Jupyter Notebook para realizar análisis de datos de pronóstico del tiempo. La idea es crear un flujo de trabajo sencillo para obtener datos para una ciudad específica usando una API de Python llamada PyOWM , ejecutar la disputa de datos, crear algunos gráficos y organizar la información en un informe en PDF.

## Trabajar con la API meteorológica de PyOWM

Como se describe en la página principal de la biblioteca, "PyOWM es una biblioteca contenedora de Python cliente para las API web de OpenWeatherMap". Facilita el acceso a los datos meteorológicos al ofrecer un "modelo de objeto simple". El único requisito para usar esta biblioteca es una clave API que está disponible de forma gratuita en el enlace de OpenWeather .

## Flujo de trabajo, parte 1: obtención de datos meteorológicos con la API de PyOWM

La primera parte del flujo de trabajo consiste en usar la biblioteca PyOWM para obtener información sobre un archivo predefinido city(Sao Paulo, BR en el cuaderno original). Iteramos sobre el forecastobjeto, organizando la información devuelta en DataFrame para hacernos la vida más fácil en los siguientes pasos.

In [1]:
# import libraries
import pyowm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pdfkit

In [2]:
# Set API key
owm = pyowm.OWM('bb453ffed6ecf2076bb80c0da53e4373')

In [3]:
# Define the default parameters
city = 'Quito,EC'

In [4]:
# Instantiate forecast object and fetch weather information about city
fc = owm.three_hours_forecast(city)
forecast = fc.get_forecast()

AttributeError: 'OWM' object has no attribute 'three_hours_forecast'

In [ ]:
# Create a dictionary to organize the forecast data for the city of interest
dict_forecast = {
    'datetime':[],
    'clouds':[],
    'humidity':[],
    'temp':[],
    'temp_max':[],
    'temp_min':[],
    'detailed_status':[],
    'icon_url':[],
    'rain_vol':[]
}

In [ ]:
#Itereate over forecast object acessing the weather features
for weather in forecast:
    dict_forecast['datetime'].append(str(weather.get_reference_time(timeformat='iso')))
    dict_forecast['clouds'].append(weather.get_clouds())
    dict_forecast['humidity'].append(weather.get_humidity())
    dict_forecast['temp'].append(weather.get_temperature(unit='celsius').get('temp'))
    dict_forecast['temp_max'].append(weather.get_temperature(unit='celsius').get('temp_max'))
    dict_forecast['temp_min'].append(weather.get_temperature(unit='celsius').get('temp_min'))
    
    dict_forecast['detailed_status'].append(weather.get_detailed_status())
    dict_forecast['icon_url'].append(weather.get_weather_icon_url())
    if '3h' in weather.get_rain().keys():
        dict_forecast['rain_vol'].append(weather.get_rain().get('3h'))
    else:
        dict_forecast['rain_vol'].append(0)

In [ ]:
# Create Dataframe from dictionary
df = pd.DataFrame.from_dict(dict_forecast)
df.head()

In [ ]:
# Plotting the temperature for the next 5 days
fig = plt.figure()
sns_plot = sns.lineplot(data=df_temp, style="event",markers=True, dashes=False)
sns_plot.set_title(f'Temperature forecast for the next 5 days', fontsize=20)
sns_plot.set_xlabel('Date', fontsize=14)
sns_plot.set_ylabel('Temperature Celsius', fontsize=14)
sns_plot.set_xticklabels(df_temp.index, rotation=20)
sns_plot.grid(True)

sns_plot.legend(labels=['Min. Temperature', 'Max Temperature', 'Average Temperature'])
fig.set_size_inches(12, 6)

temperature_plot = f"{city.split(',')[0].replace(' ','_')}_temperature.png"
sns_plot.figure.savefig(temperature_plot, 
                  dpi=300, facecolor='w', 
                  orientation='portrait',
                  bbox_inches='tight')

In [ ]:
# Create a Dataframe with total expected volume for rain on each day
df_rain_per_day = df.resample('D', on='datetime').sum()[['rain_vol']]
df_rain_per_day.index = df_rain_per_day.index.date

In [ ]:
fig = plt.figure()
# Lineplot for humidity and clouds
ax1 = fig.add_subplot(211)
ax1 = sns.lineplot(data=df_mean[['clouds', 'humidity']], markers=True, dashes=False)
ax1.set_xticks([])
ax1.set_title(f'Expected humidity and rain volume for the next 5 days', fontsize=20)
ax1.set_ylabel('Percentage', fontsize=14)
ax1.grid(True)

# Barplot for total rain per day
ax2 = fig.add_subplot(212)
ax2 = sns.barplot(x=df_rain_per_day.index, y='rain_vol', 
                  data=df_rain_per_day,
                  palette="Blues_d")
ax2.set_xticklabels(df_temp.index, rotation=30)
ax2.set_ylabel('Total Rain Volume in mm', fontsize=14)
ax2.set_xlabel('Date', fontsize=14)
fig.set_size_inches(12, 6)

rain_humidity_plot = f"{city.split(',')[0].replace(' ','_')}_rain_humidity.png"
fig.savefig(rain_humidity_plot, 
              dpi=300, facecolor='w', 
              orientation='portrait',
              bbox_inches='tight')

# Creación de un informe meteorológico en PDF

Vamos a utilizar el pdfkit de la biblioteca para convertir nuestra plantilla HTML en un archivo pdf.

In [ ]:
# Defining start and end date for the analysis
today = str(df_mean.index.min()).replace('-', '/')
last_day = str(df_mean.index.max()).replace('-', '/')

In [ ]:
# HTML template to add our data and plots
report_template = f'''
<!DOCTYPE html>
    <html>
      <head>
        <meta charset='utf-8'>
        <title>Weather Forecast with PyOWM</title>
        <link rel='stylesheet' href='report.css'>
          <style>
          h1 {{
          font-family: Arial;
          font-size: 300%;
          }}
          h2 {{
          font-family: Arial;
          font-size: 200%;
          }}
          @page {{
          size: 7in 9.25in;
          margin: 27mm 16mm 27mm 16mm;
          }}
          </style>                       
      </head>
      <h1 align="center">Weather forecast for {city}</h1>
      <h2 align="center">Initial date: {today}</h2>
      <h2 align="center">Final date: {last_day}</h2>
        
      <figure>
        <img src="{temperature_plot}" width="1200" height="600">
      </figure>
      <figure>
        <img src="{rain_humidity_plot}" width="1200" height="600">
      </figure>      
    </html>
'''

In [ ]:
# Save HTML string to file
html_report = f"{city.split(',')[0].replace(' ','_')}_report.html"
with open(html_report, "w") as r:
    r.write(report_template)

In [ ]:
# Use pdfkit to create the pdf report from the 
pdfkit.from_file(html_report, f"{city.split(',')[0].replace(' ', '_')}_weather_report_for.pdf")

# Ejecución de Papermill

# 6) Para ejecutar Papermill desde la terminal
 papermill weather_forecast_using_pyowm.ipynb \ 
          weather_forecast_using_pyowm_output.ipynb \ 
          -p city 'Sao Paulo, BR' \ 
          -k papermill-tutorial